In [20]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

def getResult(seq, yymmdd = None) :

    resultPrefix = "TEOS_TO_SON_RL_RESULT_"

    if yymmdd is None :
         yymmdd = datetime.today().strftime("%Y%m%d")

    nm = resultPrefix + str(yymmdd) + "_[" + str(seq) + "].dat"

    try:
        df = pd.read_csv(nm, sep = '|')
        print(nm)

    except FileNotFoundError:
        pass

    return(df)


def getSiteInfo() :
    df = pd.read_csv('site_info.csv', encoding = 'utf-8', header = 1) 
    df = df[['SISUL Code', 'NBR(1st).1']]
    df.columns = ['SISUL_CD1', 'NBR_CD1']
    df = df.dropna(how ='any')
    return df


def getStateQuality(seq, yymmdd = None, wa, wb) :

    df = getResult(seq, yymmdd)

    # Cell SINR avg , 5%
    def q5(x):
        return x.quantile(0.05)

    f = {'SINR1': ['mean', q5], 'RSRP1': ['mean']}
    sqResult = df.groupby('SISUL_CD1').agg(f)
    sqResult['sq'] = sqResult[('SINR1', 'mean')] * wa + (1-wa) * sqResult[('SINR1', 'q5')]

    # Nbr Cell

    def getNbrSq(x) :

        nbrSite = getSiteInfo()
        tmp = list(sqResult.index)

        nbrCell = list(nbrSite.NBR_CD1[nbrSite.SISUL_CD1 == x])

        if len(nbrCell) == 1 :
            nbrSq = sqResult.sq[tmp.index(nbrCell[0])]
        else :
            nbrSq = 99999 # nbrCell 없는 경우

        return nbrSq

    sqResult['nbrSq'] = list(map(lambda i: getNbrSq(i), sqResult.index.to_list()))

    # Global SQ
    # sqResult['gsq'] = sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')]
    sqResult['gsq'] = np.where(sqResult['nbrSq'] == 99999, sqResult['sq'] , sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')])

    sqResult.columns =  ['SINR1_Mean', 'SINR1_Q5', 'RSRP1_Mean', 'sq', 'nbrSq', 'gsq']
    sqResult['SISUL_CD'] = list(sqResult.index)


    return sqResult



SyntaxError: non-default argument follows default argument (<ipython-input-20-beb14c6df87e>, line 33)

In [21]:
os.chdir('/home/tangosvc/code/01_sumin')
os.getcwd() 

NameError: name 'os' is not defined

In [21]:
getStateQuality(11,yymmdd = None, wa = 0.5, wb = 0.5)

TEOS_TO_SON_RL_RESULT_20191018_[11].dat


KeyError: 'SISUL_CD1'